In [35]:
from datetime import datetime, timedelta
import pandas_datareader as pdr
from finta import TA
import pandas as pd

In [36]:
def get_eod_data(start_date, end_date, tickers):
    """
    Get EOD data for a list of tickers
    """    
    eod_df = pd.DataFrame([])

    for t in tickers:
        results = pdr.DataReader(t, 'yahoo', start_date, end_date)
        results['ticker'] = t  
        eod_df = eod_df.append(results)
        
    return eod_df

In [37]:
def compute_analytics(df, indicators):
    
    """ 
    Get calcs for each ticker in the df
    """
    
    data = None
    
    for indicator in indicators:

        # Using eval so we can iterate over a list of indicators 
        data = eval('TA.' + indicator + '(df)')
        
        # TA returns a set -> so we convert that to a data frame
        data = data.to_frame()    

        # The columns names are too long so I split the string 
        # and join that with the indicator name for my indicator column name 
        first_string = data.columns[0].split()[0]
        data = data.rename(columns={ data.columns[0]: first_string })    
        data = data.add_prefix(indicator + '_')

        # Merge data frames based on the date
        df = df.merge(data, left_index=True, right_index=True)
        
    return df

In [38]:
if __name__ == '__main__':
    
    start = datetime.now().date() - timedelta(days=20)
    end = datetime.now().date()
    ticker_list = ['AAPL', 'TSLA']
    eod_data = get_eod_data(start, end, ticker_list)
    
    indicator_list = ['SMA', 'EMA']
    analytics_data = eod_data.groupby(['ticker']).apply(lambda x: compute_analytics(x, indicator_list)).reset_index()

In [39]:
# Update the column names to follow PEP8
analytics_data = analytics_data.rename(columns={'Date':'date', 'High': 'high', 'Low': 'low', 'Open': 'open', 
                                                'Close': 'close', 'Volume': 'vol', 'Adj Close': 'adj_close',
                                               'SMA_41': 'sma_41', 'EMA_9': 'ema_9'})

In [40]:
analytics_data.loc[analytics_data['ticker'] == 'AAPL']

,date,high,low,open,close,vol,adj_close,ticker,sma_41,ema_9
0,2020-10-15,121.199997,118.150002,118.720001,120.709999,112559200,120.709999,AAPL,NaN,120.709999
2,2020-10-16,121.550003,118.809998,121.279999,119.019997,115393800,119.019997,AAPL,NaN,119.771109
4,2020-10-19,120.419998,115.660004,119.959999,115.980003,120639300,115.980003,AAPL,NaN,118.217377
6,2020-10-20,118.980003,115.629997,116.199997,117.510002,124423700,117.510002,AAPL,NaN,117.977751
8,2020-10-21,118.709999,116.449997,116.669998,116.870003,89946000,116.870003,AAPL,NaN,117.648221
10,2020-10-22,118.040001,114.589996,117.449997,115.750000,101988000,115.750000,AAPL,NaN,117.133698
12,2020-10-23,116.550003,114.279999,116.389999,115.040001,82572600,115.040001,AAPL,NaN,116.603839
14,2020-10-26,116.550003,112.879997,114.010002,115.050003,111850700,115.050003,AAPL,NaN,116.230423
16,2020-10-27,117.279999,114.540001,115.489998,116.599998,92276800,116.599998,AAPL,NaN,116.315797
18,2020-10-28,115.430000,111.099998,115.050003,111.199997,143937800,111.199997,AAPL,NaN,115.169561
